In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt 
from sklearn import preprocessing

from xgboost import XGBClassifier
from catboost import CatBoostClassifier, cv, Pool
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# read train and test dataset

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")

# check data state

In [ ]:
cols_with_missing = [col for col in train.columns if train[col].isnull().any()]
print(cols_with_missing)
cols_with_missing = [col for col in test.columns if test[col].isnull().any()]
print(cols_with_missing)

train and test have no missing value

In [ ]:
s = (train.dtypes == 'object')
object_cols = list(s[s].index)

s_ = (test.dtypes == 'object')
object_cols_ = list(s[s].index)

print("train Categorical variables:")
print(object_cols)
print("test Categorical variables:")
print(object_cols_)

train and test have integer only

In [ ]:
print(train.shape,test.shape)

for r in range(7):
    ct = (train['Cover_Type'] == r+1) 
    ratio = ct.sum()*100/ len(train)
    print('Cover_Type=',str(r+1),' is :',ct.sum(),'    ratio is:',ratio,'%')

kind of Cover_Type is 7 and Cover_Type='2' is most

In [ ]:
train.drop(train.index[train['Cover_Type'] == 5],inplace = True)

In [ ]:
train.describe().T

i will drop colums 'Soil_Type7' and 'Soil_Type15' because they have only '0'

In [ ]:
train.drop(['Soil_Type7','Soil_Type15'],axis = 1,inplace = True)
test.drop(['Soil_Type7','Soil_Type15'],axis = 1,inplace = True)
print(train.shape,test.shape)

# make some columns group 
continuous and categorical, in categorical i will make 2groups depends on columns name

In [ ]:
number_colums = [ 'Elevation','Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
categorical_colums1 = ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3','Wilderness_Area4'] 
categorical_colums2 = ['Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6',  'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14',  'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']

**add some more feature columns**

In [ ]:
train["f_c1"] = train[categorical_colums1].sum(axis=1)
train["f_c2"] = train[categorical_colums2].sum(axis=1)


test["f_c1"] = test[categorical_colums1].sum(axis=1)
test["f_c2"] = test[categorical_colums2].sum(axis=1)


In [ ]:
mm = preprocessing.MinMaxScaler()

# mm.fit_transform(l)
# ValueError: Expected 2D array, got 1D array instead:
train_a=train.drop(['Id','Cover_Type'],axis = 1)
l_2d_min_max = mm.fit_transform(train_a)
df = pd.DataFrame(data=l_2d_min_max, columns=train_a.columns)
l_2d_min_max_t = mm.fit_transform(test[number_colums])
df_t = pd.DataFrame(data=l_2d_min_max_t, columns=number_colums)

train["max"] = df.max(axis=1)
train["std"] = df.std(axis=1)
train["mean"] = df.mean(axis=1)
train["sum"] = df.sum(axis=1)

test["max"] = df_t.max(axis=1)
test["std"] = df_t.std(axis=1)
test["mean"] = df_t.mean(axis=1)
test["sum"] = df_t.sum(axis=1)

**prepare training data**

In [ ]:
X=train.drop(['Id','Cover_Type'],axis = 1)
y = train.Cover_Type

#Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

# create model with using XGBClassifier

In [ ]:
model_classifier = XGBClassifier(objective='multi:softmax',eval_metric = 'mlogloss')
model_classifier.fit(X_train, y_train)

In [ ]:
predict_classifier = model_classifier.predict(X_valid)
y_reshape=np.ravel(y_valid)
c = 0
e = 0
ct1 = 0
ct2 = 0
ct3 = 0
ct4 = 0
ct5 = 0
ct6 = 0
ct7 = 0
ct0 = 0
gerrorlist = []

for r in range(y_reshape.size):
        if y_reshape[r]==predict_classifier[r]:
            c = c + 1
        if y_reshape[r]!=predict_classifier[r]:
            gerrorlist.append(r)
            e = e + 1
            if y_reshape[r] == 1:
                ct1 = ct1 + 1
            elif y_reshape[r] == 2:
                ct2 = ct2 + 1
            elif y_reshape[r] == 3:
                ct3 = ct3 + 1
            elif y_reshape[r] == 4:
                ct4 = ct4 + 1
            elif y_reshape[r] == 5:
                ct5 = ct5 + 1
            elif y_reshape[r] == 6:
                ct6 = ct6 + 1
            elif y_reshape[r] == 7:
                ct7 = ct7 + 1
            else:
                ct0 = ct0 + 1

                
ratio1 = ct1*100/e
ratio2 = ct2*100/e
ratio3 = ct3*100/e
ratio5 = ct5*100/e
ratio4 = ct4*100/e
ratio6 = ct6*100/e
ratio7 = ct7*100/e

print('correct count is',c)
print('error count is',e)
print('1 is :',ct1 ,'    ratio is:',ratio1,'%')
print('2 is :',ct2 ,'    ratio is:',ratio2,'%')
print('3 is :',ct3 ,'    ratio is:',ratio3,'%')
print('4 is :',ct4 ,'    ratio is:',ratio4,'%')
print('5 is :',ct5 ,'    ratio is:',ratio5,'%')
print('6 is :',ct6 ,'    ratio is:',ratio6,'%')
print('7 is :',ct7 ,'    ratio is:',ratio7,'%')

In [ ]:
plt.figure(figsize=(10,20))

importances = pd.Series(model_classifier.feature_importances_, index = X.columns)
importances = importances.sort_values()
importances.plot(kind = "barh")
plt.title("importance in the XGBClassifier Model")

plt.show()

# create model with using CatBoostClassifier

In [ ]:
model_catclassifier = CatBoostClassifier(use_best_model = True, random_seed = 42)
model_catclassifier.fit(X_train, y_train, eval_set= (X_valid, y_valid), early_stopping_rounds=100,verbose = False)

In [ ]:
predict_classifier = model_catclassifier.predict(X_valid)
y_reshape=np.ravel(y_valid)
c = 0
e = 0
ct1 = 0
ct2 = 0
ct3 = 0
ct4 = 0
ct5 = 0
ct6 = 0
ct7 = 0
ct0 = 0
cerrorlist = []

for r in range(y_reshape.size):
        if y_reshape[r]==predict_classifier[r]:
            c = c + 1
        if y_reshape[r]!=predict_classifier[r]:
            gerrorlist.append(r)
            e = e + 1
            if y_reshape[r] == 1:
                ct1 = ct1 + 1
            elif y_reshape[r] == 2:
                ct2 = ct2 + 1
            elif y_reshape[r] == 3:
                ct3 = ct3 + 1
            elif y_reshape[r] == 4:
                ct4 = ct4 + 1
            elif y_reshape[r] == 5:
                ct5 = ct5 + 1
            elif y_reshape[r] == 6:
                ct6 = ct6 + 1
            elif y_reshape[r] == 7:
                ct7 = ct7 + 1
            else:
                ct0 = ct0 + 1

ratio1 = ct1*100/e
ratio2 = ct2*100/e
ratio3 = ct3*100/e
ratio5 = ct5*100/e
ratio4 = ct4*100/e
ratio6 = ct6*100/e
ratio7 = ct7*100/e

print('correct count is',c)
print('error count is',e)
print('1 is :',ct1 ,'    ratio is:',ratio1,'%')
print('2 is :',ct2 ,'    ratio is:',ratio2,'%')
print('3 is :',ct3 ,'    ratio is:',ratio3,'%')
print('4 is :',ct4 ,'    ratio is:',ratio4,'%')
print('5 is :',ct5 ,'    ratio is:',ratio5,'%')
print('6 is :',ct6 ,'    ratio is:',ratio6,'%')
print('7 is :',ct7 ,'    ratio is:',ratio7,'%')

In [ ]:
plt.figure(figsize=(10,20))

importances = pd.Series(model_catclassifier.feature_importances_, index = X.columns)
importances = importances.sort_values()
importances.plot(kind = "barh")
plt.title("importance in the CatBoostClassifier Model")

plt.show()

# From the above, I think

1. catboost is better than xgboost a littele bit　
2. predict error ratio with 'Cover_Type' is almost same at catboost and xgboost
3. it is interesting most important columns is same 'Elevation' but second , third or other colums is different order. and they have almost same prediction(=99.8% is same)

# check train and test data situation at Top6 importance columns in catboost model

In [ ]:
cont_features = ['Elevation','Horizontal_Distance_To_Fire_Points','Horizontal_Distance_To_Roadways', 'Wilderness_Area3','f_c2','max']
print("Feature distribution of important columns : ")
ncols = 3
nrows = 2

fig, axes = plt.subplots(nrows, ncols, figsize=(40,10), facecolor='#EAEAF2')

for r in range(nrows):
    for c in range(ncols):
        col = cont_features[r*ncols+c]
        sns.kdeplot(x=train[col], ax=axes[r, c], color='red', label='Train data')
        sns.kdeplot(x=test[col], ax=axes[r, c], color='blue', label='Test data')
        axes[r, c].set_ylabel('')
        axes[r, c].set_xlabel(col, fontsize=15, fontweight='bold')
        axes[r, c].tick_params(labelsize=10, width=0.5)
        axes[r, c].xaxis.offsetText.set_fontsize(20)
        axes[r, c].yaxis.offsetText.set_fontsize(20)
plt.show()

train and test data is almost same at important columns in catboost model so it will work to predict testdata

In [ ]:
test_target = test.drop(['Id'],axis = 1)
int_predict = model_catclassifier.predict(test_target)

In [ ]:
int_predict = np.ravel(int_predict)

In [ ]:
output = pd.DataFrame({'Id': test.Id, 'Cover_Type': int_predict})
output.to_csv('submission.csv', index=False)